# Exploratory Data Analysis

### Questions to answer:
- what features to I want to test for predictivity?
    - win percentage, SOS, 2FG%, 3FG%, 3PA%, FT%, FTM/possession, points/possession, blocks, steals, TO/possession and allowed for all these stats
    - momentum: how have you performed lately?
    - home/away/neutral
    - conference
    - experience and size (pull off of ESPN or something)
    - pace

- which features specifically predict for playoffs?
- which features are necessary to encode the performance of a team in all environments?


- train model on all historical data
    - use all reg season stats to predict winner in playoff games

1. calculate each stat for each team in each game in each season
    - decide how to calculate pace, momentum, and experience
    - figure out where to get experience, size, and conference data
2. create data.csv with each feature for each of 2 teams, home & away, with label W or L for that tourney game
    - if that's not enough data, then we can train on reg season games too with an additional variable of num_games_played
3. test feature importance
4. test many models


